# Propuesta de preguntas

## 1) Popularidad y evolución (muy defendibles)

- ¿Qué artistas mantuvieron una alta popularidad durante más años dentro del rango 2000–2005?
(no solo el más escuchado, sino el más constante)

- ¿Qué género tuvo el mayor crecimiento de popularidad a lo largo del período seleccionado?

- ¿Qué artistas debutaron en esos años y alcanzaron alta popularidad rápidamente?

👉 Aquí se puede cruzar:

año

artista

playcount / listeners

género

## B) Relación entre artistas (last.fm brilla aquí)

- ¿Qué artistas son más influyentes según el número de artistas similares asociados a ellos?

- ¿Existen artistas que conectan varios géneros a través de artistas similares?
(artistas “puente” entre géneros)

- ¿Qué artistas aparecen con más frecuencia como “similar” a otros artistas populares?

👉 Esto es mucho más original que “top artistas”.

### C) Álbumes + tiempo (sin ir a nivel canción)

- ¿Qué artistas lanzaron más álbumes en el período seleccionado y cómo se relaciona con su popularidad?

- ¿Hay relación entre la frecuencia de lanzamientos y el número de oyentes?

- ¿En qué año se concentran los álbumes más populares del período?

## D) Contenido “cualitativo” (biografías)

- ¿Los artistas con biografías más extensas tienden a ser más populares?
(hipótesis interesante, no trivial)

- ¿Qué géneros presentan artistas con biografías más detalladas?

👉 Esto demuestra que no todo es número, también texto.

In [ ]:
# Para hacer pausas (útil si hacemos muchas peticiones seguidas)
import time                                 
import requests
# Para type hints (ayuda a entender qué devuelve cada función)
from typing import Any, Dict, Optional

BASE_URL = "https://ws.audioscrobbler.com/2.0/"
API_KEY = '****************'

1) Creamos una función reutilizable para llamar a last.fm de forma estándar.

2) Construimos un diccionario con parámetros obligatorios:

  - `method`, `api_key`, `format`

3) Añadimos parámetros específicos (`artist`, `autocorrect`, etc.) usando `**params`.

4) Comprobamos errores:

  - Errores HTTP con `raise_for_status()´

  - Errores “internos” del JSON de last.fm con `if "error" in data`

In [21]:
def lastfm_request(method: str, params: Dict[str, Any], api_key: str = API_KEY, timeout: int = 30) -> Dict[str, Any]:
    """
    Función genérica para llamar a la API de Last.fm y devolver el JSON como diccionario.
    
    - method: nombre del método/endpoint de last.fm (ej: 'artist.getInfo')
    - params: parámetros específicos del método (ej: {'artist': 'Coldplay'})
    - api_key: la clave de la API
    - timeout: segundos máximos que esperamos antes de cancelar la petición
    """

    # Construimos todos los parámetros que necesita last.fm en una sola variable
    full_params = {
        "method": method,     # Qué método vamos a usar (ej: artist.getInfo)
        "api_key": api_key,   # Nuestra API Key
        "format": "json",     # Queremos la respuesta en formato JSON
        **params              # Aquí metemos el resto de parámetros específicos (artist, autocorrect, etc.)
    }

    # Hacemos la petición GET a la URL base, pasando los parámetros
    r = requests.get(BASE_URL, params=full_params, timeout=timeout)

    # Si el servidor responde con error HTTP (404, 401, 500...), aquí salta una excepción
    r.raise_for_status()

    # Convertimos la respuesta a JSON (en Python: dict/list)
    data = r.json()

    # A veces last.fm devuelve error "dentro" del JSON aunque el HTTP sea 200 (OK)
    if isinstance(data, dict) and "error" in data:
        raise ValueError(f"Last.fm API error {data.get('error')}: {data.get('message')}")

    # Si todo va bien, devolvemos el JSON
    return data


In [22]:
# Llamamos al método artist.getInfo para pedir información del artista "Coldplay"
data = lastfm_request(
    "artist.getInfo",
    {"artist": "Coldplay", "autocorrect": 1}  # autocorrect=1 corrige nombres mal escritos
)

# Vemos qué claves (keys) tiene el JSON principal de respuesta
data.keys()

dict_keys(['artist'])

In [ ]:
# Extraemos el bloque "artist" del JSON (si no existe, usamos {} para evitar errores)
artist = data.get("artist", {})

# Vemos qué información trae dentro el bloque del artista
artist.keys()

('Coldplay',
 {'listeners': '8927551', 'playcount': '763500916'},
 dict_keys(['links', 'published', 'summary', 'content']))

In [ ]:
# Extraemos la parte "artist" del JSON
# Usamos {} como valor por defecto por si no existiera la clave
artist = data.get("artist", {})

# Mostramos todas las claves que contiene el bloque del artista
artist.keys()

dict_keys(['name', 'mbid', 'url', 'image', 'streamable', 'ontour', 'stats', 'similar', 'tags', 'bio'])

## ¿Qué nos devuelve la API aquí?

`artist` es un diccionario con toda la información del artista.

Las claves más importantes para el proyecto son:

`name`: nombre del artista

`stats`: estadísticas de popularidad

`bio`: biografía del artista

`similar`: artistas similares

Otras claves (`image`, `ontour`, `streamable`) existen pero no son relevantes para nuestro análisis.

📌 Con esto confirmamos que last.fm nos da más información a nivel artista que a nivel canción, lo que refuerza nuestra decisión de usar el artista como entidad principal.

In [11]:
# Obtenemos las estadísticas del artista (oyentes y reproducciones)
artist.get("stats", {}), artist.get("bio", {}).keys()

({'listeners': '8927551', 'playcount': '763500916'},
 dict_keys(['links', 'published', 'summary', 'content']))

¿Qué información contiene cada bloque?

`stats`

 - `listeners`: número de oyentes únicos

 - `playcount`: número total de reproducciones

📌 Ambos valores vienen como strings, por lo que más adelante habrá que convertirlos a `int` para poder analizarlos en SQL.

`bio`

 - `summary`: resumen corto de la biografía

 - `content`: biografía completa

 - `published`: fecha de publicación de la biografía

📌 Esto nos permite hacer análisis cualitativos, no solo numéricos.

In [12]:
{
    # Nombre del artista
    "name": artist.get("name"),

    # Estadísticas de popularidad
    "listeners": artist.get("stats", {}).get("listeners"),
    "playcount": artist.get("stats", {}).get("playcount"),

    # Comprobamos si existen textos de biografía
    "bio_has_summary": "summary" in artist.get("bio", {}),
    "bio_has_content": "content" in artist.get("bio", {}),
}

{'name': 'Coldplay',
 'listeners': '8927551',
 'playcount': '763500916',
 'bio_has_summary': True,
 'bio_has_content': True}

¿Qué estamos haciendo aquí?

 - Estamos filtrando el JSON enorme de la API.

 - Nos quedamos solo con la información útil para el proyecto.

 - Comprobamos si la biografía existe (`True / False`) para evitar errores más adelante.

📌 Este paso es clave porque:

 - La API devuelve muchos datos.

 - Nosotras queremos estructurar la información pensando en la base de datos.

In [ ]:
from typing import Any, Dict

def get_artist_info(artist_name: str, autocorrect: int = 1) -> Dict[str, Any]:
    """
    Pide a la API de Last.fm información de un artista y devuelve un diccionario "limpio"
    y siempre con las mismas claves (aunque falten datos).
    
    - artist_name: nombre del artista que queremos buscar
    - autocorrect: si vale 1, Last.fm intenta corregir nombres mal escritos
    """

    # 1) Llamamos al endpoint/método "artist.getInfo" de Last.fm
    #    Le pasamos como parámetros el nombre del artista y autocorrect
    data = lastfm_request(
        "artist.getInfo",
        {"artist": artist_name, "autocorrect": autocorrect}
    )

    # 2) Dentro del JSON, la info principal viene en la clave "artist"
    #    Usamos {} por defecto para evitar errores si no existiera
    artist = data.get("artist", {}) or {}

    # 3) Dentro de artist, las estadísticas están dentro de la clave "stats"
    stats = artist.get("stats", {}) or {}

    # 4) La biografía está dentro de la clave "bio"
    bio = artist.get("bio", {}) or {}

    # 5) Devolvemos un diccionario con los datos que nos interesan
    #    - Convertimos listeners y playcou

¿Qué hace esta función y por qué está así?

 - Llama a la API de Last.fm usando el método `artist.getInfo`.

 - La API devuelve un JSON grande, y nosotros lo transformamos en un diccionario pequeño y ordenado.

 - `listeners` y `playcount` vienen como texto (string), por eso los convertimos a `int` para poder analizarlos en SQL.

 - Usamos `.get(..., {})` y `or {}` para evitar errores si alguna parte no viene en la respuesta.

In [28]:
# Probamos la función con un artista conocido
# Si funciona, debe devolver un diccionario con:
# name, mbid, url, listeners, playcount y bio
get_artist_info("Coldplay")

{'artist_name': 'Coldplay',
 'artist_mbid': 'cc197bad-dc9c-440d-a5b5-d52ba2e14234',
 'artist_url': 'https://www.last.fm/music/Coldplay',
 'listeners': 8927551,
 'playcount': 763500916,
 'bio_published': '02 Feb 2006, 02:58',
 'bio_summary': 'Coldplay is a British alternative rock and britpop band formed in London in 1997. They consist of vocalist and pianist Chris Martin, guitarist Jonny Buckland, bassist Guy Berryman, drummer Will Champion and creative director Phil Harvey. They met at University College London and began playing music together from 1996 to 1998, initially calling themselves Starfish. Coldplay\'s music incorporates elements of soft rock, pop rock, piano rock, and post-britpop. <a href="https://www.last.fm/music/Coldplay">Read more on Last.fm</a>',
 'bio_content': 'Coldplay is a British alternative rock and britpop band formed in London in 1997. They consist of vocalist and pianist Chris Martin, guitarist Jonny Buckland, bassist Guy Berryman, drummer Will Champion and c

¿Qué respuesta nos da la API aquí?

 - La función devuelve un diccionario ya “normalizado”.

 - Ejemplo de campos que vemos:

  - `artist_name`: nombre del artista (ya corregido si autocorrect estaba activado)

  - `listeners`: oyentes únicos

  - `playcount`: reproducciones totales

  - `bio_summary` y `bio_content`: biografía en texto (resumen y completa)

In [23]:
# Probamos con un nombre mal escrito para ver si last.fm lo corrige automáticamente
# autocorrect=1 intenta arreglar el nombre y devolver el artista correcto
get_artist_info("Colplay", autocorrect=1)

{'artist_name': 'Coldplay',
 'artist_mbid': 'cc197bad-dc9c-440d-a5b5-d52ba2e14234',
 'artist_url': 'https://www.last.fm/music/Coldplay',
 'listeners': 8927551,
 'playcount': 763500916,
 'bio_published': '02 Feb 2006, 02:58',
 'bio_summary': 'Coldplay is a British alternative rock and britpop band formed in London in 1997. They consist of vocalist and pianist Chris Martin, guitarist Jonny Buckland, bassist Guy Berryman, drummer Will Champion and creative director Phil Harvey. They met at University College London and began playing music together from 1996 to 1998, initially calling themselves Starfish. Coldplay\'s music incorporates elements of soft rock, pop rock, piano rock, and post-britpop. <a href="https://www.last.fm/music/Coldplay">Read more on Last.fm</a>',
 'bio_content': 'Coldplay is a British alternative rock and britpop band formed in London in 1997. They consist of vocalist and pianist Chris Martin, guitarist Jonny Buckland, bassist Guy Berryman, drummer Will Champion and c

¿Qué estamos comprobando aquí?

 - Que aunque el usuario escriba mal el nombre del artista, la API puede corregirlo si activamos `autocorrect=1`.

Esto es útil para evitar errores y para hacer el sistema más “amigable” con entradas reales.

In [24]:
from typing import List

def get_similar_artists(artist_name: str, limit: int = 10, autocorrect: int = 1) -> List[Dict[str, Any]]:
    """
    Devuelve una lista de artistas similares para un artista dado.
    Cada elemento de la lista es un diccionario con info del artista similar.
    """
    data = lastfm_request(
        "artist.getSimilar",
        {
            "artist": artist_name,
            "limit": limit,
            "autocorrect": autocorrect
        }
    )

    # La lista suele venir dentro de: data["similarartists"]["artist"]
    similar_block = data.get("similarartists", {}) or {}
    similar_list = similar_block.get("artist", []) or []

    results = []

    # Recorremos cada artista similar y normalizamos lo que nos interesa
    for a in similar_list:
        results.append({
            "source_artist": artist_name,
            "similar_artist_name": a.get("name"),
            "similar_artist_mbid": a.get("mbid") or None,
            "similar_artist_url": a.get("url") or None,
            # "match" suele venir como string tipo "0.87" -> lo convertimos a float
            "match": float(a["match"]) if a.get("match") else None
        })

    return results


¿Qué nos devuelve la API y qué guardamos?

 - Este endpoint devuelve una lista de artistas “parecidos”.

 - Nosotros lo normalizamos en una lista de diccionarios con:

  - `source_artist`: artista original (el que estamos consultando)

  - `similar_artist_name`: nombre del artista similar

  - `match`: nivel de similitud (lo convertimos a float para análisis)

In [27]:
similar = get_similar_artists("Coldplay", limit=5)
similar

[{'source_artist': 'Coldplay',
  'similar_artist_name': 'Keane',
  'similar_artist_mbid': 'c7020c6d-cae9-4db3-92a7-e5c561cbad50',
  'similar_artist_url': 'https://www.last.fm/music/Keane',
  'match': 1.0},
 {'source_artist': 'Coldplay',
  'similar_artist_name': 'Imagine Dragons',
  'similar_artist_mbid': '012151a8-0f9a-44c9-997f-ebd68b5389f9',
  'similar_artist_url': 'https://www.last.fm/music/Imagine+Dragons',
  'match': 0.781353},
 {'source_artist': 'Coldplay',
  'similar_artist_name': 'OneRepublic',
  'similar_artist_mbid': 'c33c2065-b1c3-4406-b066-d33a9e2ea71a',
  'similar_artist_url': 'https://www.last.fm/music/OneRepublic',
  'match': 0.725816},
 {'source_artist': 'Coldplay',
  'similar_artist_name': 'Snow Patrol',
  'similar_artist_mbid': 'a66999a7-ae5c-460e-ba94-1a01143ae847',
  'similar_artist_url': 'https://www.last.fm/music/Snow+Patrol',
  'match': 0.628487},
 {'source_artist': 'Coldplay',
  'similar_artist_name': 'Travis',
  'similar_artist_mbid': '22a40b75-affc-4e69-8884-2